In [1]:
import os
import google.generativeai as genai
from PIL import Image
import mimetypes

/Users/natalyagrokh/miniconda3/envs/img_curation_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Configuration ---
# Set your Google Cloud project details here.
# These are necessary for the SDK to target the Vertex AI API.
GCP_PROJECT_ID = "MLexpressImgSorting"
GCP_LOCATION = "us-central1"  # e.g., "us-central1"

In [3]:
# Set the path to your service account key file.
# The SDK will automatically use this for authentication.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/natalyagrokh/AI/img_curation/mlexpimgsorting-v2-c5a570b110c3.json"

In [4]:
def perform_inpainting(image_path: str, prompt: str):
    """
    Uses the Gemini 1.5 Pro model on Vertex AI to perform image inpainting.

    Args:
        image_path: The local file path to the image.
        prompt: The text prompt describing the inpainting task.

    Returns:
        The generated content response from the model.
    """
    print(f"Starting inpainting process for: {image_path}")

    try:
        # 1. Configure the SDK to use Vertex AI
        # This is the key step to ensure the library calls the correct enterprise-grade endpoint.
        genai.configure(
            project=GCP_PROJECT_ID,
            location=GCP_LOCATION,
        )

        # 2. Load the model
        # We specify "gemini-1.5-pro" as required.
        model = genai.GenerativeModel("gemini-1.5-pro")

        # 3. Prepare the image for the API call
        # The SDK can handle PIL Image objects directly.
        print("Opening image...")
        image = Image.open(image_path)

        # 4. Construct the multimodal prompt
        # The prompt consists of both the instructional text and the image.
        print("Sending request to Vertex AI API...")
        contents = [prompt, image]

        # 5. Make the API call
        # The generate_content method handles the complex HTTP request and authentication.
        response = model.generate_content(contents)

        print("Successfully received response from the model.")
        return response

    except Exception as e:
        print(f"\n--- An Error Occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        print("\n--- Troubleshooting ---")
        print("1. Verify that the GOOGLE_APPLICATION_CREDENTIALS path is correct.")
        print("2. Ensure the Vertex AI API is enabled in your GCP project.")
        print("3. CRITICAL: If you are seeing a '403 Forbidden' or 'PermissionDenied' error,")
        print("   this is likely due to a VPC Service Controls perimeter. The code is correct,")
        print("   but your network security is blocking the call. Contact your cloud administrator")
        print("   and request an 'ingress rule' to allow your machine's IP to access the")
        print("   'aiplatform.googleapis.com' service.")
        return None


In [5]:
if __name__ == "__main__":
    # --- Example Usage ---
    # Replace with the actual path to your image and your desired prompt
    example_image_path = "face_image.png"
    example_prompt = "This is an image of a person's face. Perform inpainting to remove the glasses."

    # Create a dummy image file for testing if it doesn't exist
    if not os.path.exists(example_image_path):
        print(f"Creating a dummy image file at '{example_image_path}' for demonstration.")
        try:
            dummy_image = Image.new('RGB', (100, 100), color = 'red')
            dummy_image.save(example_image_path)
        except Exception as e:
            print(f"Could not create dummy image: {e}")
            exit()

    # Run the main function
    api_response = perform_inpainting(example_image_path, example_prompt)

    if api_response:
        # The response object contains multiple parts; we print the text part.
        # For image generation tasks, you would process other parts of the response.
        print("\n--- Model Response ---")
        print(api_response.text)
        print("----------------------")

Starting inpainting process for: face_image.png

--- An Error Occurred ---
Error Type: TypeError
Error Details: configure() got an unexpected keyword argument 'project'

--- Troubleshooting ---
1. Verify that the GOOGLE_APPLICATION_CREDENTIALS path is correct.
2. Ensure the Vertex AI API is enabled in your GCP project.
3. CRITICAL: If you are seeing a '403 Forbidden' or 'PermissionDenied' error,
   this is likely due to a VPC Service Controls perimeter. The code is correct,
   but your network security is blocking the call. Contact your cloud administrator
   and request an 'ingress rule' to allow your machine's IP to access the
   'aiplatform.googleapis.com' service.
